In [ ]:
import json
import time
import datetime
import re
import os
import sys
import subprocess
import pandas as pd
import pytesseract
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image
import base64
import cv2 as cv
from itertools import zip_longest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import threading
    
def request_url(no):
    
    global result
    
    chrome_options = Options()
    chrome_options.add_argument('headless') #无界面
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) #防检测
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_window_size(1920,1080)  #设置分辨率

    cookies = {
        'domain': 'avedex.cc',
        'expiry': 1662347549,
        'httpOnly': False,
        'name': 'ave_token',
        'path': '/',
        'secure': False,
        'value': '133561fcb642e3599bd1b79f6abd33331661742728595159144'
        }

    driver.get("https://avedex.cc/market")
    time.sleep(3)
    driver.add_cookie(cookie_dict = cookies)
    driver.get("https://avedex.cc/market")
    # print('===》正在打开页面')
    time.sleep(3)

    # print('===>正在获取中')
    page_value = f'/html/body/div[1]/main/section/div[2]/div/div[1]/div[3]/div/div[1]/div/table/tbody/tr[{no}]'
    driver.implicitly_wait(30)  #隐式等待5秒
    coin = driver.find_element(by=By.XPATH,value=page_value).text.split('\n')  #代币信息->列表
    ans = ['序号','币种','最新价格','24h涨幅','流通量','开盘价','风险性']
    res = dict(zip_longest(ans,coin))

    #跳转到详细页面
    page_value = f'/html/body/div[1]/main/section/div[2]/div/div[1]/div[3]/div/div[1]/div/table/tbody/tr[{no}]'
    driver.implicitly_wait(30)  #隐式等待5秒
    driver.find_element(by=By.XPATH,value=page_value).click()

    #点击info
    time.sleep(2)
    page_value = '/html/body/div[1]/main/section/div[1]/div[4]/section/div[1]/div[1]/div/div[2]'
    driver.implicitly_wait(30)  #隐式等待5秒
    driver.find_element(by=By.XPATH,value=page_value).click()

    #代币合约&所属链
    current_url = driver.current_url
    contract = re.search('(?<=token/).*?(?=-)',current_url).group()
    chain = re.search('(?<=-).*',current_url).group()
    res['合约'] = contract
    res['所属链'] = chain

    #相关链接
    links = []
    try:
        page_value = '/html/body/div[1]/main/section/div[1]/div[4]/section/div[2]/div[1]/div/ul/li'
        length = len(driver.find_elements(by=By.XPATH,value=page_value))
        page_value = f'/html/body/div[1]/main/section/div[1]/div[4]/section/div[2]/div[1]/div/ul/li[{length}]/div/a'
        for i in driver.find_elements(by=By.XPATH,value=page_value):
            link = i.get_attribute('href')
            links.append(link)
    except:
        pass
    res['相关链接'] = links

    lock.acquire()    #线程锁
    result.append(res)    
    lock.release()    #释放锁
    
    print(f'{no}爬取完成')
    driver.close()
    pool_sema.release() #释放已完成线程
        
        
    
result = []

lock = threading.Lock()  #设置线程锁
max_connection = 4 #4线程
pool_sema = threading.BoundedSemaphore(max_connection) #最大线程量

thread_list = []
for i in tqdm(range(1,11)):
    pool_sema.acquire()
    thread = threading.Thread(target=request_url,args=[i])
    thread.start()
    thread_list.append(thread)

for t in thread_list:
    t.join()

date_ = datetime.datetime.now().strftime('%Y-%m-%d %H')   #更新时间
df = pd.DataFrame(result)
df['序号'] = df['序号'].map(lambda i:int(i))
df['更新时间'] = date_
df = df.sort_values('序号')
df.to_csv(f'./save/ave{date_}更新.csv',index=False)

In [1]:
%run avedex.py

 10%|████████▎                                                                          | 1/10 [00:14<02:09, 14.42s/it]

1爬取完成


 20%|████████████████▌                                                                  | 2/10 [00:24<01:33, 11.72s/it]

2爬取完成


 40%|█████████████████████████████████▏                                                 | 4/10 [01:02<01:34, 15.72s/it]

...

8爬取完成


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:52<00:10, 10.82s/it]

6爬取完成
9爬取完成


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:41<00:00, 16.13s/it]


10爬取完成


In [2]:
%run MyToken.py

 50%|█████████████████████████████████████████▌                                         | 5/10 [00:27<00:27,  5.46s/it]

4爬取完成
1爬取完成


 70%|██████████████████████████████████████████████████████████                         | 7/10 [00:27<00:10,  3.56s/it]

2爬取完成


 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [00:28<00:05,  2.90s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:56<00:08,  8.66s/it]

5爬取完成


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:11<00:00,  7.16s/it]

9爬取完成


6爬取完成
8爬取完成
10爬取完成
7爬取完成
